In [1]:
import operator
import random

import numpy

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

In [2]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.Fitness, speed=list, smin=None, smax=None, best=None)

In [3]:
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size))
    
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    
    part.smin = smin
    part.smax = smax
    return part

In [4]:
toolbox = base.Toolbox()

pmin = -100
pmax = 100

toolbox.register("particle", generate, size=10, pmin=pmin, pmax=pmax, smin=pmin*0.1, smax=pmax*0.1)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)

In [5]:
def updateParticle(part, best, c1, c2):
    u1 = (random.uniform(0, c1) for _ in range(len(part)))
    u2 = (random.uniform(0, c2) for _ in range(len(part)))
    
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    
    part.speed = map(operator.add, part.speed, map(operator.add, v_u1, v_u2))
    
    inertia_weights = (random.uniform(0.4, 0.9) for _ in range(len(part)))
    
    part.speed = list(map(operator.mul, inertia_weights, part.speed))
    
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
            
    part[:] = list(map(operator.add, part, part.speed))

In [6]:
toolbox.register("update", updateParticle, c1=1.8, c2=1.8)
toolbox.register("evaluate", benchmarks.sphere)

In [7]:
def main():
    pop = toolbox.population(n=30)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 10000
    best = None
    
    all_bests = []

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
                
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        
        all_bests.append(best.fitness.values)
        
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best, all_bests

In [8]:
if __name__ == "__main__":
    results = main()
    
#     for item in results[3]:
#         print(item) 
        
    print(results[2])

gen	evals	avg    	std    	min    	max    
0  	30   	31895.6	8916.86	14637.4	54968.8
1  	30   	29063.9	8954.12	12222.2	57026.1
2  	30   	27825.4	9523.27	11157.4	58464.6
3  	30   	27802.4	10030.9	10869.6	59622.9
4  	30   	27789.7	11301.1	13069.7	60331.9
5  	30   	28280.1	12650.1	12594.2	60851.2
6  	30   	28470.1	13699.6	10912.4	61200.8
7  	30   	29541.1	14516.4	7853.65	61397.6
8  	30   	30622.4	15260.8	6539.89	61505.6
9  	30   	31996.9	15958.9	5687.4 	61600.6
10 	30   	33272.8	16280  	6265.05	61663.9
11 	30   	34901  	16821  	7967.3 	61693  
12 	30   	35754.3	17762.2	8622.67	62893  
13 	30   	36636.2	18760.3	9018.41	65312.6
14 	30   	38496.1	19776  	9120.85	67684  
15 	30   	40970.2	20815.2	11747.4	70462.3
16 	30   	43656.2	22344.5	9847.86	72068.9
17 	30   	46176.2	23975.5	10796.8	78046.2
18 	30   	48560.5	26532.3	11040.7	82393.1
19 	30   	51614.6	28602.1	11986.4	88781.3
20 	30   	55426.7	30740.2	14102.1	93154.3
21 	30   	59056.2	32815  	17641.3	96061.2
22 	30   	62208.7	34187.3	19252.1	

203	30   	2.68325e+06	12424.8	2.65376e+06	2.70522e+06
204	30   	2.71246e+06	11208.4	2.68631e+06	2.73172e+06
205	30   	2.73848e+06	10117.5	2.72134e+06	2.7597e+06 
206	30   	2.76506e+06	10533.8	2.73679e+06	2.78e+06   
207	30   	2.79008e+06	9177.94	2.76345e+06	2.80822e+06
208	30   	2.81494e+06	9920.81	2.79922e+06	2.83173e+06
209	30   	2.83994e+06	9060.49	2.82011e+06	2.8586e+06 
210	30   	2.86669e+06	7736.76	2.84802e+06	2.88291e+06
211	30   	2.89241e+06	9385.45	2.86411e+06	2.90652e+06
212	30   	2.91614e+06	9105.52	2.89669e+06	2.93199e+06
213	30   	2.94024e+06	9023.96	2.91856e+06	2.96331e+06
214	30   	2.9674e+06 	9978.51	2.93496e+06	2.98843e+06
215	30   	2.99447e+06	11453.1	2.95634e+06	3.01268e+06
216	30   	3.02048e+06	10525.7	2.99525e+06	3.03685e+06
217	30   	3.046e+06  	12064.2	3.00726e+06	3.06997e+06
218	30   	3.07345e+06	12026.9	3.03552e+06	3.0958e+06 
219	30   	3.10336e+06	9568.91	3.08206e+06	3.12435e+06
220	30   	3.13219e+06	12080.2	3.10951e+06	3.16037e+06
221	30   	3.1627e+06 	14996.

544	30   	1.92474e+07	27684.3	1.91885e+07	1.92958e+07
545	30   	1.93204e+07	27614  	1.9265e+07 	1.93674e+07
546	30   	1.93935e+07	23505.1	1.93263e+07	1.94325e+07
547	30   	1.94567e+07	22584.2	1.93858e+07	1.94988e+07
548	30   	1.95159e+07	22150.6	1.9458e+07 	1.95526e+07
549	30   	1.9573e+07 	23577.1	1.95294e+07	1.9624e+07 
550	30   	1.96338e+07	28260.6	1.95727e+07	1.96918e+07
551	30   	1.96998e+07	28570.2	1.96269e+07	1.97443e+07
552	30   	1.97632e+07	17200.7	1.97249e+07	1.98034e+07
553	30   	1.98271e+07	22859.7	1.97881e+07	1.98761e+07
554	30   	1.98979e+07	25491  	1.98421e+07	1.99538e+07
555	30   	1.99719e+07	26436.8	1.99125e+07	2.00127e+07
556	30   	2.00399e+07	25395.8	1.99547e+07	2.00803e+07
557	30   	2.0104e+07 	25712.7	2.00486e+07	2.01479e+07
558	30   	2.01624e+07	26038.6	2.01067e+07	2.02057e+07
559	30   	2.02263e+07	23532.9	2.01695e+07	2.02749e+07
560	30   	2.02932e+07	24240.5	2.02487e+07	2.03475e+07
561	30   	2.0364e+07 	25528.8	2.03146e+07	2.04198e+07
562	30   	2.04368e+07	29062.

771	30   	3.91448e+07	32167.5	3.90445e+07	3.91976e+07
772	30   	3.9231e+07 	26711.6	3.91651e+07	3.92813e+07
773	30   	3.93145e+07	25909.2	3.92529e+07	3.93604e+07
774	30   	3.93947e+07	37288.1	3.93159e+07	3.94606e+07
775	30   	3.94809e+07	40111.5	3.93843e+07	3.95433e+07
776	30   	3.95698e+07	38675.2	3.94681e+07	3.96223e+07
777	30   	3.96524e+07	30171.9	3.95755e+07	3.97084e+07
778	30   	3.97397e+07	26763.1	3.96944e+07	3.97821e+07
779	30   	3.98215e+07	24356.1	3.9777e+07 	3.9874e+07 
780	30   	3.99007e+07	31684  	3.98426e+07	3.99541e+07
781	30   	3.99829e+07	38807.6	3.98907e+07	4.00378e+07
782	30   	4.00646e+07	35346.3	3.99684e+07	4.01316e+07
783	30   	4.01428e+07	36443.9	4.00297e+07	4.02096e+07
784	30   	4.02196e+07	31722.9	4.01432e+07	4.0279e+07 
785	30   	4.02995e+07	33950.3	4.02327e+07	4.0356e+07 
786	30   	4.03779e+07	39372.7	4.03224e+07	4.04623e+07
787	30   	4.04696e+07	43545.3	4.03875e+07	4.05424e+07
788	30   	4.05586e+07	41836.2	4.04618e+07	4.06355e+07
789	30   	4.06554e+07	30712.

955	30   	5.74234e+07	41811.4	5.73258e+07	5.74897e+07
956	30   	5.75507e+07	49394.2	5.7431e+07 	5.76267e+07
957	30   	5.76774e+07	49609.2	5.75899e+07	5.77755e+07
958	30   	5.78085e+07	43577.4	5.77275e+07	5.78807e+07
959	30   	5.79291e+07	46323  	5.78317e+07	5.8038e+07 
960	30   	5.8068e+07 	43904.4	5.80125e+07	5.81999e+07
961	30   	5.82306e+07	37097.5	5.81656e+07	5.83186e+07
962	30   	5.83848e+07	40325.4	5.83107e+07	5.84612e+07
963	30   	5.85275e+07	42773  	5.84317e+07	5.86055e+07
964	30   	5.86616e+07	43524.2	5.85631e+07	5.87285e+07
965	30   	5.87788e+07	46321.4	5.86609e+07	5.88877e+07
966	30   	5.89163e+07	37849.1	5.88431e+07	5.90072e+07
967	30   	5.90477e+07	33011.2	5.89804e+07	5.91158e+07
968	30   	5.91749e+07	43039  	5.90912e+07	5.92506e+07
969	30   	5.92903e+07	50835.6	5.91863e+07	5.94012e+07
970	30   	5.94191e+07	53857.6	5.93062e+07	5.95435e+07
971	30   	5.95577e+07	56527.1	5.94407e+07	5.96426e+07
972	30   	5.96876e+07	55861.3	5.95261e+07	5.97818e+07
973	30   	5.98206e+07	55620.

...

8995	30   	4.15199e+09	431730 	4.15118e+09	4.15282e+09
8996	30   	4.15289e+09	361664 	4.15227e+09	4.15343e+09
8997	30   	4.15366e+09	309808 	4.15302e+09	4.15437e+09
8998	30   	4.15451e+09	343434 	4.15375e+09	4.15521e+09
8999	30   	4.15539e+09	346136 	4.15454e+09	4.15602e+09
9000	30   	4.15628e+09	365607 	4.15544e+09	4.15692e+09
9001	30   	4.15714e+09	377964 	4.15599e+09	4.15768e+09
9002	30   	4.15797e+09	314279 	4.15715e+09	4.15848e+09
9003	30   	4.15879e+09	310406 	4.1583e+09 	4.15943e+09
9004	30   	4.15965e+09	323331 	4.15912e+09	4.16021e+09
9005	30   	4.16043e+09	342153 	4.15958e+09	4.161e+09  
9006	30   	4.16122e+09	305356 	4.16052e+09	4.1618e+09 
9007	30   	4.16203e+09	354435 	4.16123e+09	4.16282e+09
9008	30   	4.16291e+09	363923 	4.1618e+09 	4.16359e+09
9009	30   	4.16383e+09	391380 	4.16294e+09	4.16455e+09
9010	30   	4.16476e+09	372738 	4.1639e+09 	4.16539e+09
9011	30   	4.16567e+09	405630 	4.1647e+09 	4.16634e+09
9012	30   	4.16654e+09	357147 	4.16569e+09	4.16713e+09
9013	30   

9202	30   	4.33002e+09	391345 	4.32923e+09	4.3307e+09 
9203	30   	4.33089e+09	347125 	4.32995e+09	4.33148e+09
9204	30   	4.33168e+09	304563 	4.3308e+09 	4.33211e+09
9205	30   	4.33242e+09	310622 	4.33174e+09	4.33295e+09
9206	30   	4.33311e+09	398474 	4.33211e+09	4.33378e+09
9207	30   	4.3339e+09 	453894 	4.33288e+09	4.33477e+09
9208	30   	4.33475e+09	481505 	4.33352e+09	4.33579e+09
9209	30   	4.33571e+09	530857 	4.33428e+09	4.33665e+09
9210	30   	4.33667e+09	429261 	4.33546e+09	4.33729e+09
9211	30   	4.33752e+09	397119 	4.3364e+09 	4.33817e+09
9212	30   	4.33837e+09	411107 	4.33759e+09	4.33921e+09
9213	30   	4.33933e+09	422416 	4.33868e+09	4.34018e+09
9214	30   	4.34029e+09	444493 	4.33941e+09	4.34101e+09
9215	30   	4.34117e+09	513580 	4.34001e+09	4.34194e+09
9216	30   	4.34205e+09	444991 	4.34117e+09	4.34262e+09
9217	30   	4.34283e+09	416223 	4.34175e+09	4.34355e+09
9218	30   	4.34365e+09	488335 	4.34235e+09	4.34438e+09
9219	30   	4.34444e+09	566149 	4.34307e+09	4.34524e+09
9220	30   

9424	30   	4.52327e+09	374631 	4.52237e+09	4.52398e+09
9425	30   	4.52422e+09	378930 	4.52306e+09	4.52482e+09
9426	30   	4.52509e+09	450114 	4.52364e+09	4.5258e+09 
9427	30   	4.52599e+09	476244 	4.52486e+09	4.52663e+09
9428	30   	4.52685e+09	377620 	4.52608e+09	4.52753e+09
9429	30   	4.52775e+09	350941 	4.52687e+09	4.52843e+09
9430	30   	4.52859e+09	465209 	4.52752e+09	4.52931e+09
9431	30   	4.52946e+09	456795 	4.52823e+09	4.53025e+09
9432	30   	4.53038e+09	438458 	4.52931e+09	4.53118e+09
9433	30   	4.5313e+09 	438047 	4.53047e+09	4.53194e+09
9434	30   	4.53222e+09	440123 	4.53136e+09	4.53289e+09
9435	30   	4.53316e+09	426891 	4.53162e+09	4.53391e+09
9436	30   	4.53405e+09	511135 	4.53172e+09	4.53451e+09
9437	30   	4.53489e+09	456052 	4.53294e+09	4.5355e+09 
9438	30   	4.53571e+09	405909 	4.53416e+09	4.53634e+09
9439	30   	4.53658e+09	377306 	4.53537e+09	4.53733e+09
9440	30   	4.53745e+09	430556 	4.53648e+09	4.53817e+09
9441	30   	4.53838e+09	444633 	4.53725e+09	4.53906e+09
9442	30   

9664	30   	4.74114e+09	339118 	4.74038e+09	4.74188e+09
9665	30   	4.74202e+09	403769 	4.74097e+09	4.7427e+09 
9666	30   	4.74286e+09	461115 	4.74176e+09	4.74349e+09
9667	30   	4.74376e+09	414034 	4.74279e+09	4.74441e+09
9668	30   	4.74462e+09	363685 	4.74357e+09	4.74522e+09
9669	30   	4.74546e+09	392095 	4.7445e+09 	4.74611e+09
9670	30   	4.74629e+09	396327 	4.74559e+09	4.7472e+09 
9671	30   	4.7473e+09 	355404 	4.74661e+09	4.74803e+09
9672	30   	4.74827e+09	360794 	4.74737e+09	4.74894e+09
9673	30   	4.7492e+09 	411117 	4.74817e+09	4.74982e+09
9674	30   	4.75007e+09	403052 	4.74868e+09	4.75071e+09
9675	30   	4.75088e+09	472915 	4.74914e+09	4.7515e+09 
9676	30   	4.75168e+09	446861 	4.75039e+09	4.75232e+09
9677	30   	4.7525e+09 	388552 	4.75165e+09	4.75317e+09
9678	30   	4.75341e+09	392916 	4.75255e+09	4.75408e+09
9679	30   	4.75428e+09	428856 	4.75313e+09	4.75509e+09
9680	30   	4.75518e+09	478084 	4.75352e+09	4.75606e+09
9681	30   	4.75616e+09	496370 	4.75473e+09	4.75696e+09
9682	30   

9873	30   	4.93324e+09	623913 	4.93126e+09	4.93403e+09
9874	30   	4.93419e+09	523244 	4.93254e+09	4.93477e+09
9875	30   	4.93505e+09	450850 	4.93382e+09	4.93577e+09
9876	30   	4.93588e+09	385994 	4.9351e+09 	4.93674e+09
9877	30   	4.93681e+09	287757 	4.93598e+09	4.93731e+09
9878	30   	4.93764e+09	371062 	4.93677e+09	4.93827e+09
9879	30   	4.93849e+09	451816 	4.93737e+09	4.93906e+09
9880	30   	4.93935e+09	454838 	4.93814e+09	4.9401e+09 
9881	30   	4.94022e+09	482739 	4.93921e+09	4.94094e+09
9882	30   	4.94109e+09	517942 	4.93959e+09	4.94184e+09
9883	30   	4.94201e+09	491791 	4.94072e+09	4.9427e+09 
9884	30   	4.9429e+09 	462692 	4.942e+09  	4.94368e+09
9885	30   	4.9438e+09 	470386 	4.94268e+09	4.94451e+09
9886	30   	4.94464e+09	525026 	4.94345e+09	4.94564e+09
9887	30   	4.94559e+09	630674 	4.94391e+09	4.94657e+09
9888	30   	4.94664e+09	637856 	4.94499e+09	4.94744e+09
9889	30   	4.94763e+09	546889 	4.94626e+09	4.94849e+09
9890	30   	4.94867e+09	503463 	4.94754e+09	4.94947e+09
9891	30   